# Лабораторная работа по теме Проблемы собственных значений

### Вариант 5
Степенной метод

##### описание метода
это алгоритм для нахождения 

In [120]:
import numpy as np

In [121]:
def find_spectral_radius(
    matrix: np.ndarray,
    tol: float = 1e-10, 
    max_iter: int = 10000,
) -> tuple[float, np.ndarray]:
    """
    Находит спектральный радиус матрицы (максимальное по модулю собственное значение)
    с помощью степенного метода.
    """
    
    """
    этап 1
    инициализируем результат A^kx, чтобы наблюдать сходимость
    создаём случайный вектор x
    и нормируем, для избежания переполнения
    """
    checker_prev = 0
    x = np.random.randn(matrix.shape[0])
    x = x / np.linalg.norm(x)
    
    """
    этап 2
    начинаем итерироваться
    """
    for iteration in range(max_iter):
        """
        продолжаем домножать на A и отслеживать насколько изменилась матрица
        ждём ситуации, когда изменений происходить не будет
        то есть
        (l - лямбда)
        x_0 = c_1*v_1+...+c_n*v_n
        A^k*x_0 = l_1^k*c_1*v_1+...+l_n^k*c_n*v_n
        A^k*x_0 = l_1^k((l_1^k/l_1^k)*c_1*v_1+...+(l_n^k/l_1^k)*c_n*v_n)
        и т.к. |l1| > |l_i| для всех i
        (для i>1) (l_i/l_1) -> 0
        A^k*x_0 -> l_1^k(c_1*v_1)
        тогда A^{k+1}*x_0 -> l_1*l_1^k(c_1*v_1)
        и если мы заметим (A^{k+2}*x_0)/(A^{k+1}*x_0)==(A^{k+1}*x_0)/(A^{k}*x_0), 
        то это будет признаком, что
        (A^{k+2}*x_0)/(A^{k+1}*x_0)==(A^{k+1}*x_0)/(A^{k}*x_0)==l_1
        """
        y = matrix @ x

        checker_curr = (x @ y) / (x @ x) if np.linalg.norm(x) > 1e-15 else 0

        y_norm = np.linalg.norm(y)
        if y_norm < 1e-15:
            # Если вектор стал нулевым, значит собственное значение 0
            checker_curr = 0
            break
        x = y / y_norm
        
        # Проверка сходимости
        if iteration > 0 and abs(checker_curr - checker_prev) < tol:
            break
            
        checker_prev = checker_curr
    
    else:
        print(f"Предупреждение: не достигнута точность {tol} за {max_iter} итераций")
    
    spectral_radius = checker_curr
    
    return spectral_radius, x


In [122]:
def test(A: np.ndarray) -> None:
    print(f"исходная матрица {A}")
    max_self_val, self_vec = find_spectral_radius(A)
    np_max, np_min = np.max(np.linalg.eigvals(A)), np.min(np.linalg.eigvals(A))
    max_self_val_np = np_max if np_max > abs(np_min) else np_min
    print(
        f"мой спектральный радиус: {max_self_val:.15f}",
        f"точный спектральный радиус: {max_self_val_np:.15f}",
        f"Ошибка: {abs(max_self_val - max_self_val_np):.6e}",
        sep="\n"
    )
    print("-"*50)
    print(
        f"собственный вектор моего спектрального радиуса {self_vec}",
        "Проверка A*v = l*v",
        f"Норма разности: {np.linalg.norm(A @ self_vec - max_self_val * self_vec):.2e}",
        sep="\n"
    )

In [ ]:
# Тест: Простая симметричная матрица
A = np.array([
    [2, 1],
    [1, 2],
], dtype=float)
test(A)

исходная матрица [[2. 1.]
 [1. 2.]]
мой спектральный радиус: 2.999999999997160
точный спектральный радиус: 3.000000000000000
Ошибка: 2.839506e-12
--------------------------------------------------
собственный вектор моего спектрального радиуса [-0.70710706 -0.7071065 ]
Проверка A*v = l*v
Норма разности: 7.94e-07


In [ ]:
# Тест: Матрица с отрицательным максимальным собственным значением
A = np.array([
    [-3, 1],
    [1, 1]
], dtype=float)
test(A)

исходная матрица [[-3.  1.]
 [ 1.  1.]]
мой спектральный радиус: -3.236067977489436
точный спектральный радиус: -3.236067977499790
Ошибка: 1.035350e-11
--------------------------------------------------
собственный вектор моего спектрального радиуса [-0.97324912  0.22975235]
Проверка A*v = l*v
Норма разности: 2.60e-06


In [ ]:
# Тест: Случайная матрица
SIZE = 3
np.random.seed(42)
A = np.random.randn(SIZE, SIZE)
test(A)

исходная матрица [[ 0.49671415 -0.1382643   0.64768854]
 [ 1.52302986 -0.23415337 -0.23413696]
 [ 1.57921282  0.76743473 -0.46947439]]
мой спектральный радиус: 1.227956864193705
точный спектральный радиус: 1.227956864312035+0.000000000000000j
Ошибка: 1.183300e-10
--------------------------------------------------
собственный вектор моего спектрального радиуса [0.54229542 0.45141488 0.70861858]
Проверка A*v = l*v
Норма разности: 4.90e-10


In [ ]:
# Тест: Диагональная матрица
A = np.diag([1, 5, 3, 2])  # Спектральный радиус = 5
test(A)

исходная матрица [[1 0 0 0]
 [0 5 0 0]
 [0 0 3 0]
 [0 0 0 2]]
мой спектральный радиус: 4.999999999963503
точный спектральный радиус: 5.000000000000000
Ошибка: 3.649703e-11
--------------------------------------------------
собственный вектор моего спектрального радиуса [ 4.24344871e-19 -1.00000000e+00 -2.56313266e-06 -3.30886956e-11]
Проверка A*v = l*v
Норма разности: 5.13e-06
